<a href="https://colab.research.google.com/github/Radomir21/Data-Analysis-2024/blob/main/Lab_4_Besh_Radomyr_Advanced_Nets__ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.  Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір). Вирішіть завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.**

In [17]:
#імпорти бібліотек
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization, MultiHeadAttention, LayerNormalization, Dense, Embedding, Dropout
from tensorflow.keras.models import Model
import re
import string
import random
from transformers import pipeline
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
text_file = '/content/drive/MyDrive/Data Analyzis/ukr.txt'

In [6]:
with open(text_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

text_pairs = []
for line in lines:
    eng, ukr, _ = line.strip().split("\t")
    ukr = "[start] " + ukr + " [end]"
    text_pairs.append((eng, ukr))

In [7]:
for _ in range(5):
    print(random.choice(text_pairs))

("Who's that guy, anyway?", '[start] Хто цей чувак взагалі такий? [end]')
("Don't look for us.", '[start] Не шукай нас. [end]')
('Mr. Jackson is our principal.', '[start] Пан Джексон — наш директор. [end]')
("I'll do whatever it takes to get you back into my life.", '[start] Я зроблю все необхідне, щоб ти повернулася в моє життя. [end]')
('Tom is looking for an apartment.', '[start] Том шукає квартиру. [end]')


In [8]:

random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]


print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

159432 total pairs
111604 training pairs
23914 validation pairs
23914 test pairs


In [9]:
vocab_size = 15000
sequence_length = 20

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

eng_vectorization = TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length)
ukr_vectorization = TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length + 1, standardize=custom_standardization)

train_eng_texts = [pair[0] for pair in train_pairs]
train_ukr_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
ukr_vectorization.adapt(train_ukr_texts)

In [13]:
train_eng_texts[:100]

["I wonder how much we'll have to pay.",
 'I have mixed emotions about this.',
 'I have a few ideas.',
 'Eat a lot of vegetables.',
 'You should wear a coat.',
 'She lives in the village.',
 "Tom's uncle sent him a Christmas card.",
 "You've seen worse.",
 'I got very hungry.',
 'I went to the park to play tennis.',
 'I taught Tom French three years ago.',
 'I did everything I could do.',
 'Are you thirsty?',
 'Come sit next to me.',
 "Don't vote for it.",
 'I know that Tom will try to do that.',
 'My pen is new.',
 'Who do you like the most?',
 "Go home, Tom. You're drunk.",
 'I think that we need to help Tom.',
 "I've already paid you.",
 'Tom wants to talk to you again.',
 'Everyone loves football.',
 "I'm very disappointed in you.",
 'Tom and Mary are nice.',
 "We're at the bank.",
 'I want to be able to speak French.',
 'Tom sells carpets.',
 'You know what I think.',
 'That apple is big.',
 'How was Tom killed?',
 'Why did Tom decide to study French?',
 'Tom is a hypocrite.',
 'W

In [14]:
train_ukr_texts[:100]

['[start] Цікаво, скільки ми муситимо заплатити. [end]',
 '[start] У мене змішані почуття щодо цього. [end]',
 '[start] Я маю кілька ідей. [end]',
 '[start] Їж багато овочів. [end]',
 '[start] Ти повинен вдягнути плащ. [end]',
 '[start] Вона живе в селі. [end]',
 '[start] Дядько Тома відправив йому різдвяну листівку. [end]',
 '[start] Ви бачили й гірше. [end]',
 '[start] Я дуже зголодніла. [end]',
 '[start] Я пішов до парку пограти в теніс. [end]',
 '[start] Я викладав Тому французьку три роки тому. [end]',
 '[start] Я зробила все, що могла. [end]',
 '[start] Ти хочеш пити? [end]',
 '[start] Йди сядь поруч зі мною. [end]',
 '[start] Не голосуйте за це. [end]',
 '[start] Я знаю, що Том спробує це зробити. [end]',
 '[start] Моя ручка нова. [end]',
 '[start] Хто тобі подобається більше за всіх? [end]',
 "[start] Йди додому, Томе. Ти п'яний. [end]",
 '[start] Гадаю, нам потрібно допомогти Тому. [end]',
 '[start] Я тобі вже заплатила. [end]',
 '[start] Том знову хоче поговорити з тобою. [en

In [15]:
class TransformerEncoderLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.dense_proj(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [16]:
class TransformerDecoderLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.attention1 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention2 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.dropout3 = Dropout(rate)

    def call(self, inputs, enc_output, training):
        attn_output1 = self.attention1(inputs, inputs)
        attn_output1 = self.dropout1(attn_output1, training=training)
        out1 = self.layernorm1(inputs + attn_output1)
        attn_output2 = self.attention2(out1, enc_output)
        attn_output2 = self.dropout2(attn_output2, training=training)
        out2 = self.layernorm2(out1 + attn_output2)
        ffn_output = self.dense_proj(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)


In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = layers.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = Embedding(input_dim=vocab_size, output_dim=embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoderLayer(embed_dim, num_heads, latent_dim)(x, training=True)

decoder_inputs = layers.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = layers.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = Embedding(input_dim=vocab_size, output_dim=embed_dim)(decoder_inputs)
x = TransformerDecoderLayer(embed_dim, num_heads, latent_dim)(x, encoder_outputs, training=True)
decoder_outputs = Dense(vocab_size, activation="softmax")(x)

transformer = Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")


transformer.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


def format_dataset(eng, ukr):
    eng = eng_vectorization(eng)
    ukr = ukr_vectorization(ukr)
    return ({"encoder_inputs": eng, "decoder_inputs": ukr[:, :-1]}, ukr[:, 1:])


epochs = 3


batch_size = 128

def make_dataset(pairs, batch_size):
    eng_texts, ukr_texts = zip(*pairs)
    dataset = tf.data.Dataset.from_tensor_slices((list(eng_texts), list(ukr_texts)))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

train_ds = make_dataset(train_pairs, batch_size)
val_ds = make_dataset(val_pairs, batch_size)

transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)


Epoch 1/3
868/868 [==============================] - 4870s 6s/step - loss: 1.4914 - accuracy: 0.8074 - val_loss: 3.1767 - val_accuracy: 0.7121
Epoch 2/3
868/868 [==============================] - 4865s 6s/step - loss: 1.1184 - accuracy: 0.8466 - val_loss: 0.6312 - val_accuracy: 0.8908
Epoch 3/3
868/868 [==============================] - 4868s 6s/step - loss: 0.6907 - accuracy: 0.8894 - val_loss: 0.4837 - val_accuracy: 0.9088


In [ ]:
ukr_vocab = ukr_vectorization.get_vocabulary()
ukr_index_lookup = dict(zip(range(len(ukr_vocab)), ukr_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ukr_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ukr_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence.replace("[start]", "").strip()


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(f"EN: {input_sentence}\nUKR: {translated}\n")

# Example translation
print("Example translation of 'hi':")
print(decode_sequence('hi'))


EN: Is this book yours?
UKR: Так твоя книжка [end]

EN: I'm surprised at your behavior.
UKR: Боюся ваш улюблений напій [end]

EN: Let him do it.
UKR: Дайно ним [end]

EN: Learning a foreign language is a waste of time.
UKR: Час вивчати мови [end]

EN: Nobody saw anything.
UKR: Ніхто нічого бачив [end]

Example translation of 'hi':
Привіт Томе [end]


# **2. Проведіть експерименти з моделями бібліотеки Hugging Face (раніше - Hugging Face Transformers,) за допомогою (наприклад) Pipeline модуля**

In [ ]:
pip install transformers

In [ ]:
pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline


uk_models = [
    "csebuetnlp/mT5_multilingual_XLSum",
    "Geotrend/bert-base-uk-cased",
    "IlyaGusev/rut5_base_sum_gazeta_uk",
    "sberbank-ai/ruRoberta-large"
]

# Створення пайплайнів
pipelines = {}
for model in uk_models:
    try:
        if "sum" in model:
            pipelines[model] = pipeline("summarization", model=model)
        else:
            pipelines[model] = pipeline("fill-mask", model=model)
    except Exception as e:
        print(f"Не вдалося створити пайплайн для моделі {model}: {e}")

# Приклад використання пайплайну для сумаризації
text_for_summarization = "Новий рік принесе щастя і мир"
try:
    summarized_text = pipelines["csebuetnlp/mT5_multilingual_XLSum"](text_for_summarization)
    print("Сумаризований текст:", summarized_text)
except Exception as e:
    print(f"Помилка при сумаризації тексту: {e}")

# Приклад використання пайплайну для заповнення пропущених слів
text_for_fill_mask = "Вітаю з [MASK] роком"
try:
    filled_text = pipelines["Geotrend/bert-base-uk-cased"](text_for_fill_mask)
    print("Текст із заповненими пропусками:", filled_text)
except Exception as e:
    print(f"Помилка при заповненні пропущених слів: {e}")



Не вдалося створити пайплайн для моделі csebuetnlp/mT5_multilingual_XLSum: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.
Не вдалося створити пайплайн для моделі IlyaGusev/rut5_base_sum_gazeta_uk: IlyaGusev/rut5_base_sum_gazeta_uk is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Помилка при сумаризації тексту: 'csebuetnlp/mT5_multilingual_XLSum'
Текст із заповненими пропусками: [{'score': 0.26641497015953064, 'token': 10709, 'token_str': 'новим', 'sequence': 'Вітаю з новим роком'}, {'score': 0.2534734308719635, 't

# **3. Завдання щодо генерації або стилізації зображень (на вибір) Вирішіть завдання перенесення стилю або генерації зображень (архітектура за вашим вибором: GAN/DCGAN/VAE/Diffusion).**

In [ ]:
pip install torch torchvision


In [18]:
batch_size = 128
image_size = 28
nz = 100
ngf = 64
ndf = 64
num_epochs = 5
lr = 0.0002
beta1 = 0.5


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # State size. (ngf*4) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # State size. (ngf*2) x 7 x 7
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # State size. (ngf) x 14 x 14
            nn.ConvTranspose2d(ngf, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # State size. (1) x 28 x 28
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input size. (1) x 28 x 28
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State size. (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # State size. (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, 1, 7, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)


netG = Generator().to(device)
netD = Discriminator().to(device)


criterion = nn.BCELoss()


fixed_noise = torch.randn(64, nz, 1, 1, device=device)


real_label = 1
fake_label = 0


optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):

        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()


        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label).type(torch.float)
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()


        netG.zero_grad()
        label.fill_(real_label).type(torch.float)
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Check how the generator is doing by saving G's output on fixed_noise
        if (epoch % 1 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            save_image(fake, f'output_epoch_{epoch}.png', nrow=8, normalize=True)

print("Training Finished.")



100%|██████████| 26.4M/26.4M [00:03<00:00, 8.70MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 132kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 2.44MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 11.4MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

[0/5][0/469]	Loss_D: 1.3299	Loss_G: 1.1541	D(x): 0.4626	D(G(z)): 0.4194 / 0.3219
[0/5][50/469]	Loss_D: 0.0882	Loss_G: 3.9089	D(x): 0.9567	D(G(z)): 0.0402 / 0.0211
[0/5][100/469]	Loss_D: 0.9872	Loss_G: 3.9822	D(x): 0.8342	D(G(z)): 0.5208 / 0.0265
[0/5][150/469]	Loss_D: 0.8381	Loss_G: 1.8900	D(x): 0.7597	D(G(z)): 0.4101 / 0.1657
[0/5][200/469]	Loss_D: 0.5141	Loss_G: 2.3175	D(x): 0.7942	D(G(z)): 0.2346 / 0.1074
[0/5][250/469]	Loss_D: 0.6325	Loss_G: 1.7137	D(x): 0.6651	D(G(z)): 0.1679 / 0.1936
[0/5][300/469]	Loss_D: 0.5901	Loss_G: 1.6594	D(x): 0.7411	D(G(z)): 0.2281 / 0.2111
[0/5][350/469]	Loss_D: 0.7232	Loss_G: 2.4543	D(x): 0.8396	D(G(z)): 0.4046 / 0.0994
[0/5][400/469]	Loss_D: 0.7390	Loss_G: 1.0463	D(x): 0.5994	D(G(z)): 0.1546 / 0.3758
[0/5][450/469]	Loss_D: 0.7026	Loss_G: 2.6981	D(x): 0.8720	D(G(z)): 0.4161 / 0.0815
[1/5][0/469]	Loss_D: 0.6530	Loss_G: 1.9614	D(x): 0.7967	D(G(z)): 0.3307 / 0.1560
[1/